In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from src.features import build_features

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION = [f"Poblacion {i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

In [3]:
ruta_base_de_datos = (
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
)

# Lee la base de datos
df = pd.read_excel(ruta_base_de_datos).iloc[:-2]

# Renombra columna de hombres o mujeres
df = df.rename(columns={"Sexo\n1=Hombre\n2=Mujer": "hombre_mujer"})

# Indica si es adulto o infantil
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO

In [4]:
ESTRATOS_A_CALCULAR = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR["SSMO"] = df_ssmo

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

In [5]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "pais": "", # Todo el pais
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios

}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.

In [6]:
# Obtiene resultados por estratos y grupos etarios
results = build_features.iterate_queries(ESTRATOS_A_CALCULAR, query_strings, COLUMNAS_POBLACION)
# Guarda resultados en un unico excel
build_features.multiple_dfs(results, "../data/interim/poblacion_INE_estratos.xlsx", 1)

---

In [17]:
RUTA_PLANILLA_INCIDENCIAS = "../data/processed/proyeccion_problemas_de_salud.xlsx"
COLUMNAS_A_UTILIZAR = ["Diagnostico", "Incidencia (Cada 100.000)", "Edad Incidencia"]

incidencias = pd.read_excel(
    RUTA_PLANILLA_INCIDENCIAS, sheet_name="incidencias_prevalencias", usecols=COLUMNAS_A_UTILIZAR
)

In [16]:
incidencias

,Diagnostico,Incidencia (Cada 100.000),Edad Incidencia
0,C33X - Tumor maligno de la tráquea,0.260,todos
1,C340 - Tumor maligno del bronquio principal,6.900,todos
2,"C341 - Tumor maligno del lóbulo superior, bron...",6.900,todos
3,"C342 - Tumor maligno del lóbulo medio, bronqui...",6.900,todos
4,"C343 - Tumor maligno del lóbulo inferior, bron...",6.900,todos
5,C381 - Tumor maligno del mediastino anterior,1.200,todos
6,C384 - Tumor maligno de la pleura,0.035,todos
7,C450 - Mesotelioma de la pleura,0.035,todos
8,C780 - Tumor maligno secundario del pulmón,6.900,todos
9,C782 - Tumor maligno secundario de la pleura,6.900,todos
